<a href="https://colab.research.google.com/github/ghundal/E115_SMART/blob/milestone3/fine_tuning/smart_gemma_fine_tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Set-up

## Install packages

In [1]:
# Install Gemma release branch from Hugging Face
%pip install git+https://github.com/huggingface/transformers@v4.49.0-Gemma-3

# Install Hugging Face libraries
%pip install  --upgrade \
  huggingface_hub \
  transformers \
  datasets \
  evaluate \
  accelerate \
  bitsandbytes \
  trl \
  peft \
  protobuf \
  wandb \
  gcsfs

# COMMENT IN: if you are running on a GPU that supports BF16 data type and flash attn, such as NVIDIA L4 or NVIDIA A100
%pip install flash-attn

#patch for gemma 3 issue
%pip install --upgrade git+https://github.com/huggingface/transformers.git

  Cloning https://github.com/huggingface/transformers (to revision v4.49.0-Gemma-3) to /tmp/pip-req-build-ughus7bx
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-ughus7bx
  Running command git checkout -q 367bab469b0ef32017e2a0a0a5dbac5d36002f03
  Resolved https://github.com/huggingface/transformers to commit 367bab469b0ef32017e2a0a0a5dbac5d36002f03
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for transformers: filename=transformers-4.50.0.dev0-py3-none-any.whl size=10936468 sha256=2cf83436b028ff0c7fee5d82929d7a295e966b2db14c588dcba9853637e676f7
  Stored in directory: /tmp/pip-ephem-wheel-cache-tme0nkx6/wheels/53/15/d5/d63b866c641d8863f9cd29a4cc7a5efc38476c3aae8247c195
Successfully built transformers
  Attempting uninstall: transformers
    Found existing installation: transformers 4.51.0.dev0
    Uninstalling t

## Package Imports

In [2]:
import os
import json
from datasets import Dataset

#google credentials
from google.colab import (
    userdata,
    auth as google_auth,
    drive as google_drive
    )
from google.cloud import storage
from google.oauth2 import service_account

# Login into Hugging Face Hub
from huggingface_hub import login

import wandb

# Model training
import torch
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    BitsAndBytesConfig,
    TrainingArguments,
    logging,
    pipeline
    )

from peft import LoraConfig, PeftModel
from trl import SFTConfig, SFTTrainer

## Environment Set-up

In [3]:
# mount google drive
google_drive.mount('/content/drive')
%cd /content/drive/MyDrive/Colab\ Notebooks/CSCI_115

# authenticate to gcp
google_auth.authenticate_user()
credentials = service_account.Credentials.from_service_account_file(
            "./secrets/smart_input_key.json"
        )
data_dir = "fine_tuning/datasets_60q/"

# Login to WandB
wandb.login(key=userdata.get("WANDB_API_KEY"))

os.environ["WANDB_LOG_MODEL"] = "checkpoint" # W&B auto-logging of models as artifacts
os.environ["WANDB_WATCH"] = "all" # W&B log params

run = wandb.init(project='smart_gemma_ft', job_type="training", anonymous="allow")

# Login into Hugging Face Hub
hf_token = userdata.get('HF_TOKEN') # If you are running inside a Google Colab
login(hf_token)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/Colab Notebooks/CSCI_115


wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: tvaldeca (tvaldeca-harvard-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


# Import Dataset

In [4]:
def connect_to_bucket():
    """Connect to the GCS bucket and return the bucket object."""
    try:
        client = storage.Client(project="SMART", credentials=credentials)
        bucket = client.bucket("smart_input_data")
        return bucket
    except Exception as e:
        raise RuntimeError(f"Critical failure: Unable to connect to GCS bucket: {str(e)}")

def download_dataset(bucket, filename):
  """Download the dataset from the bucket"""
  blob = bucket.blob(f'{data_dir}{filename}')
  blob.download_to_filename(filename)

def load_dataset(filename):
  """Load the dataset from the filename"""
  with open(filename, 'r') as f:
    dataset = Dataset.from_dict(json.load(f))
  return dataset

In [5]:
filename = "dataset_60q.json"

# get gcp bucket connection
bucket = connect_to_bucket()

# download and load dataset
download_dataset(bucket, filename)
dataset = load_dataset(filename)

# shuffle then split to train/test
dataset = dataset.shuffle().train_test_split(test_size=0.2)

# Load Gemma Model

In [6]:
# Hugging Face model id
model_id = "google/gemma-3-1b-it"

# Check if GPU benefits from bfloat16
if torch.cuda.get_device_capability()[0] >= 8:
    torch_dtype = torch.bfloat16
else:
    torch_dtype = torch.float16

# Define model init arguments
model_kwargs = dict(
    attn_implementation="eager", # Use "flash_attention_2" when running on Ampere or newer GPU
    torch_dtype=torch_dtype, # What torch dtype to use, defaults to auto
    device_map="auto", # Let torch decide how to load the model
)

# BitsAndBytesConfig: Enables 4-bit quantization to reduce model size/memory usage
model_kwargs["quantization_config"] = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_compute_dtype=model_kwargs['torch_dtype'],
    bnb_4bit_quant_storage=model_kwargs['torch_dtype'],
)

# Load model and tokenizer
model = AutoModelForCausalLM.from_pretrained(model_id, **model_kwargs)
tokenizer = AutoTokenizer.from_pretrained("google/gemma-3-1b-it") # Load the Instruction Tokenizer to use the official Gemma template

## Training Configuration

### QLoRA

In [7]:
peft_config = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.05,
    r=16,
    bias="none",
    target_modules="all-linear",
    task_type="CAUSAL_LM",
    modules_to_save=["lm_head", "embed_tokens"] # make sure to save the lm_head and embed_tokens as you train the special tokens
)

### Hyperparameters

In [8]:
args = TrainingArguments(
    output_dir="gemma-smart-fine-tuned-v2",         # directory to save and repository id
    overwrite_output_dir = True,
    eval_steps = 10,                        # perform evaluation every 10 steps
    num_train_epochs=7,                     # number of training epochs
    per_device_train_batch_size=1,          # batch size per device during training
    per_device_eval_batch_size=1,
    report_to="wandb",                      # enables logging to W&B 😎
    run_name='960_qs_7_epochs_cosine',       # W&B run name
    gradient_accumulation_steps=3,          # number of steps before performing a backward/update pass
    gradient_checkpointing=True,            # use gradient checkpointing to save memory
    optim="adamw_torch_fused",              # use fused adamw optimizer
    logging_steps=10,                       # log every 10 steps
    logging_strategy="steps",
    save_strategy="epoch",                  # save checkpoint every epoch
    learning_rate=2e-4,                     # learning rate, based on QLoRA paper
    fp16=True if torch_dtype == torch.float16 else False,   # use float16 precision
    bf16=True if torch_dtype == torch.bfloat16 else False,   # use bfloat16 precision
    max_grad_norm=0.3,                      # max gradient norm based on QLoRA paper
    warmup_ratio=0.03,                      # warmup ratio based on QLoRA paper
    lr_scheduler_type="cosine",           # use constant learning rate scheduler
    push_to_hub=True                       # push model to hub
)

# Disable caching during training for gradient computation efficiency
model.config.use_cache = False

# Training

In [9]:
# Create Trainer object
trainer = SFTTrainer(
    model=model,
    args=args,
    train_dataset=dataset['train'],
    eval_dataset=dataset['test'],
    peft_config=peft_config,
    processing_class=tokenizer
)

Map:   0%|          | 0/768 [00:00<?, ? examples/s]

Converting train dataset to ChatML:   0%|          | 0/768 [00:00<?, ? examples/s]

Applying chat template to train dataset:   0%|          | 0/768 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/768 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/768 [00:00<?, ? examples/s]

...

Converting eval dataset to ChatML:   0%|          | 0/192 [00:00<?, ? examples/s]

Applying chat template to eval dataset:   0%|          | 0/192 [00:00<?, ? examples/s]

Tokenizing eval dataset:   0%|          | 0/192 [00:00<?, ? examples/s]

Truncating eval dataset:   0%|          | 0/192 [00:00<?, ? examples/s]

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [ ]:
# Start training, the model will be automatically saved to the Hub and the output directory
trainer.train()

# Save the final model again to the Hugging Face Hub
trainer.save_model()

# tell W&B training complete
wandb.finish()
model.config.use_cache = True

wandb: WARNING Artifacts logged anonymously cannot be claimed and expire after 7 days.


Step,Training Loss
10,3.690000
20,2.594600
30,1.832100
40,1.543500
50,1.426200
60,1.343100
70,1.344400
80,1.287200
90,1.299300
100,1.261200


In [ ]:
# free the memory again
del model
del trainer
torch.cuda.empty_cache()

# Merge QLoRA Adapter Weights

In [ ]:
# Load Model base model
model = AutoModelForCausalLM.from_pretrained(model_id, low_cpu_mem_usage=True)

# Merge LoRA and base model and save
peft_model = PeftModel.from_pretrained(model, args.output_dir)
merged_model = peft_model.merge_and_unload()
merged_model.save_pretrained("merged_model", safe_serialization=True, max_shard_size="2GB")

processor = AutoTokenizer.from_pretrained(args.output_dir)
processor.save_pretrained("merged_model")

In [ ]:
# send model to gcp
bucket_name = "smart_input_data"
gcs_model_path = f"gs://{bucket_name}/fine_tuning/gemma_model/"

# send merged model
!gsutil -m cp -r {"merged_model"} {gcs_model_path}

# send fine-tuned model
!gsutil -m cp -r {"gemma-smart-fine-tuned-v2"} {gcs_model_path}

# send W&B files
!gsutil -m cp -r {"wandb"} {gcs_model_path}

# Test Model Inference

In [ ]:
question = "What is cross validation?"

messages = [{"role": "user", "content": question}]
prompt = processor.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
inputs = processor(prompt, return_tensors='pt', padding=True, truncation=True).to("cpu")

# Optimized generation with tuned sampling strategies
outputs = merged_model.generate(
    **inputs,
    max_length=350,  # Increase max length for complex answers
    num_return_sequences=1,
    top_k=50,
    top_p=0.85,  # Narrow top-p for more deterministic output
    temperature=0.3,  # Slightly higher temperature for balance between creativity and accuracy
    no_repeat_ngram_size=3,
)

# Decode and clean up the output
text = processor.decode(outputs[0], skip_special_tokens=True)
response = text.split("\nmodel\n")[1].strip()

print("\n",response)

# References

- https://ai.google.dev/gemma/docs/core/huggingface_text_finetune_qlora
- https://wandb.ai/capecape/alpaca_ft/reports/How-to-Fine-tune-an-LLM-Part-3-The-HuggingFace-Trainer--Vmlldzo1OTEyNjMy
- https://medium.com/@anicomanesh/fine-tuning-gemma-2-for-medical-question-answering-a-step-by-step-guide-1c6c4ec4c107